# Use scikit-learn and custom library to predict temperature with `ibm-watsonx-ai`

This notebook contains steps and code to train a `scikit-learn` model that uses a custom defined transformer and use it with watsonx.ai Runtime service. Once the model is trained, this notebook contains steps to persist the model and custom defined transformer to watsonx.ai Runtime, deploy and score it using `ibm-watsonx-ai` python client.

In this notebook, we use GNFUV dataset that contains mobile sensor readings data about humidity and temperature from Unmanned Surface Vehicles in a test-bed in Athens, to train a Scikit-Learn model for predicting the temperature. 

Some familiarity with Python and Bash is helpful. This notebook uses Python 3.12.

## Learning goals

The learning goals of this notebook are:

- Train a model with custom defined transformer
- Persist the custom defined transformer and the model in watsonx.ai Runtime.
- Deploy the model using watsonx.ai Runtime service
- Perform predictions using the deployed model

## Contents
1. [Set up the environment](#setup)
2. [Install python library containing custom transformer implementation](#install_lib)
3. [Prepare training data](#load)
4. [Train the scikit-learn model](#train)
5. [Save the model and library to WML Repository](#upload)
6. [Deploy and score data](#deploy)
7. [Clean up](#cleanup)
8. [Summary and next steps](#summary)


<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Contact with your Cloud Pak for Data administrator and ask them for your account credentials

### Install dependencies
**Note:** `ibm-watsonx-ai` documentation can be found <a href="https://ibm.github.io/watsonx-ai-python-sdk/index.html" target="_blank" rel="noopener no referrer">here</a>.

In [1]:
%pip install wget | tail -n 1
%pip install "scikit-learn==1.6.1" | tail -n 1
%pip install -U setuptools | tail -n 1
%pip install -U ibm-watsonx-ai | tail -n 1

#### Define credentials

Authenticate the watsonx.ai Runtime service on IBM Cloud Pak for Data. You need to provide the **admin's** `username` and the platform `url`.

In [2]:
username = "PASTE YOUR USERNAME HERE"
url = "PASTE THE PLATFORM URL HERE"

Use the **admin's** `api_key` to authenticate watsonx.ai Runtime services:

In [3]:
import getpass
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    username=username,
    api_key=getpass.getpass("Enter your watsonx.ai API key and hit enter: "),
    url=url,
    instance_id="openshift",
    version="5.2",
)

Alternatively you can use the **admin's** `password`:

In [3]:
import getpass
from ibm_watsonx_ai import Credentials

if "credentials" not in locals() or not credentials.api_key:
    credentials = Credentials(
        username=username,
        password=getpass.getpass("Enter your watsonx.ai password and hit enter: "),
        url=url,
        instance_id="openshift",
        version="5.2",
    )

#### Create `APIClient` instance

In [4]:
from ibm_watsonx_ai import APIClient

client = APIClient(credentials)

### Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have space already created, you can use `{PLATFORM_URL}/ml-runtime/spaces?context=icp4data` to create one.

- Click New Deployment Space
- Create an empty space
- Go to space `Settings` tab
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watsonx-ai-samples/blob/master/cpd5.2/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [5]:
space_id = "PASTE YOUR SPACE ID HERE"

You can use `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in watsonx.ai, you need to set **space** which you will be using.

In [6]:
client.set.default_space(space_id)

'SUCCESS'

<a id="install_lib"></a>

## 2. Install the library containing custom transformer

Library `linalgnorm-0.1` is a Python distributable package that contains the implementation of a user defined Scikit-Learn transformer - `LNormalizer`. Any 3rd party libraries that are required for the custom transformer must be defined as a dependency for the corresponding library that contains implementation of the transformer.

In this section, we will create the library and install it in the current notebook environment. 

In [7]:
!mkdir -p linalgnorm-0.1/linalg_norm

Define a custom Scikit transformer.

In [8]:
%%writefile linalgnorm-0.1/linalg_norm/sklearn_transformers.py

from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np


class LNormalizer(BaseEstimator, TransformerMixin):
    def __init__(self, norm_ord=2):
        self.norm_ord = norm_ord
        self.row_norm_vals = None

    def fit(self, X, y=None):
        self.row_norm_vals = np.linalg.norm(X, ord=self.norm_ord, axis=0)

    def transform(self, X, y=None):
        return X / self.row_norm_vals

    def fit_transform(self, X, y=None):
        self.fit(X, y)
        return self.transform(X, y)

    def get_norm_vals(self):
        return self.row_norm_vals

Writing linalgnorm-0.1/linalg_norm/sklearn_transformers.py


Wrap created code into Python source distribution package.

In [9]:
%%writefile linalgnorm-0.1/linalg_norm/__init__.py

__version__ = "0.1"

Writing linalgnorm-0.1/linalg_norm/__init__.py


In [10]:
%%writefile linalgnorm-0.1/README.md

A simple library containing a simple custom scikit estimator.

Writing linalgnorm-0.1/README.md


In [ ]:
%%writefile linalgnorm-0.1/setup.py

from setuptools import setup

VERSION="0.1"

setup(
    name="linalgnorm",
    version=VERSION,
    url="https://github.ibm.com/NGP-TWC/repository/",
    author="IBM",
    author_email="ibm@ibm.com",
    license="IBM",
    packages=[
        "linalg_norm"
    ],
    zip_safe=False
)

Writing linalgnorm-0.1/setup.py


In [12]:
%%bash

cd linalgnorm-0.1
python setup.py sdist --formats=zip
cd ..
mv linalgnorm-0.1/dist/linalgnorm-0.1.zip .
rm -rf linalgnorm-0.1

running sdist
running egg_info
creating linalgnorm.egg-info
writing linalgnorm.egg-info/PKG-INFO
writing dependency_links to linalgnorm.egg-info/dependency_links.txt
writing top-level names to linalgnorm.egg-info/top_level.txt
writing manifest file 'linalgnorm.egg-info/SOURCES.txt'
reading manifest file 'linalgnorm.egg-info/SOURCES.txt'
writing manifest file 'linalgnorm.egg-info/SOURCES.txt'
running check
creating linalgnorm-0.1
creating linalgnorm-0.1/linalg_norm
creating linalgnorm-0.1/linalgnorm.egg-info
copying files to linalgnorm-0.1...
copying README.md -> linalgnorm-0.1
copying setup.py -> linalgnorm-0.1
copying linalg_norm/__init__.py -> linalgnorm-0.1/linalg_norm
copying linalg_norm/sklearn_transformers.py -> linalgnorm-0.1/linalg_norm
copying linalgnorm.egg-info/PKG-INFO -> linalgnorm-0.1/linalgnorm.egg-info
copying linalgnorm.egg-info/SOURCES.txt -> linalgnorm-0.1/linalgnorm.egg-info
copying linalgnorm.egg-info/dependency_links.txt -> linalgnorm-0.1/linalgnorm.egg-info
copyi

Install the downloaded library using `pip` command

In [ ]:
%pip install linalgnorm-0.1.zip

<a id="load"></a>

## 3. Download training dataset and prepare training data

Download the data from UCI repository - https://archive.ics.uci.edu/ml/machine-learning-databases/00452/GNFUV%20USV%20Dataset.zip

In [14]:
!rm -rf dataset
!mkdir dataset

In [ ]:
import wget

wget.download(
    url="https://archive.ics.uci.edu/ml/machine-learning-databases/00452/GNFUV%20USV%20Dataset.zip",
    out="dataset/gnfuv_dataset.zip",
)

'dataset/gnfuv_dataset.zip'

In [16]:
!unzip dataset/gnfuv_dataset.zip -d dataset

Archive:  dataset/gnfuv_dataset.zip
  inflating: dataset/pi2/gnfuv-temp-exp1-55d487b85b-5g2xh_1.0.csv  
  inflating: dataset/pi3/gnfuv-temp-exp1-55d487b85b-2bl8b_1.0.csv  
  inflating: dataset/pi4/gnfuv-temp-exp1-55d487b85b-xcl97_1.0.csv  
  inflating: dataset/pi5/gnfuv-temp-exp1-55d487b85b-5ztk8_1.0.csv  
  inflating: dataset/README.pdf      


Create pandas datafame based on the downloaded dataset

In [17]:
import json
import pandas as pd
import numpy as np
import os
from datetime import datetime
from json import JSONDecodeError

In [18]:
home_dir = "./dataset"
pi_dirs = os.listdir(home_dir)

data_list = []
base_time = None
columns = None

for pi_dir in pi_dirs:
    if "pi" not in pi_dir:
        continue

    curr_dir = os.path.join(home_dir, pi_dir)
    data_file = os.path.join(curr_dir, os.listdir(curr_dir)[0])

    with open(data_file) as file:
        line = file.readline().strip().replace("'", '"')
        while line != "":
            try:
                input_json = json.loads(line)
                sensor_datetime = datetime.fromtimestamp(input_json["time"])

                if base_time is None:
                    base_time = datetime(
                        year=sensor_datetime.year,
                        month=sensor_datetime.month,
                        day=sensor_datetime.day,
                        hour=0,
                        minute=0,
                        second=0,
                        microsecond=0,
                    )

                input_json["time"] = (sensor_datetime - base_time).seconds
                data_list.append(list(input_json.values()))

                if columns is None:
                    columns = list(input_json.keys())
            except JSONDecodeError:
                pass

            line = file.readline().strip().replace("'", '"')

data_df = pd.DataFrame(data_list, columns=columns)

In [19]:
data_df.head()

,device,humidity,temperature,experiment,time
0,gnfuv-temp-exp1-55d487b85b-5g2xh,21.0,40.0,1.0,69557
1,gnfuv-temp-exp1-55d487b85b-5g2xh,21.0,40.0,1.0,69571
2,gnfuv-temp-exp1-55d487b85b-5g2xh,21.0,40.0,1.0,69577
3,gnfuv-temp-exp1-55d487b85b-5g2xh,21.0,40.0,1.0,69583
4,gnfuv-temp-exp1-55d487b85b-5g2xh,22.0,40.0,1.0,69589


Create training and test datasets from the downloaded GNFUV-USV dataset.

In [20]:
from sklearn.model_selection import train_test_split

y = data_df["temperature"]
X = data_df.drop("temperature", axis=1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=143
)

<a id="train"></a>

## 4. Train a model

In this section, you will use the custom transformer as a stage in the Scikit-Learn `Pipeline` and train a model.

#### Import the custom transformer 
Here, import the custom transformer that has been defined in `linalgnorm-0.1.zip` and create an instance of it that will inturn be used as stage in `sklearn.Pipeline`

In [21]:
from linalg_norm.sklearn_transformers import LNormalizer

In [22]:
lnorm_transf = LNormalizer()

Import other objects required to train a model

In [23]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

Now, you can create a `Pipeline` with user defined transformer as one of the stages and train the model

In [24]:
skl_pipeline = Pipeline(
    steps=[("normalizer", lnorm_transf), ("regression_estimator", LinearRegression())]
)
skl_pipeline.fit(X_train.loc[:, ["time", "humidity"]].values, y_train)

Pipeline(steps=[('normalizer', LNormalizer()),
                ('regression_estimator', LinearRegression())])

In [25]:
y_pred = skl_pipeline.predict(X_test.loc[:, ["time", "humidity"]].values)
rmse = np.mean((np.round(y_pred) - y_test.values) ** 2) ** 0.5
print("RMSE: {}".format(rmse))

RMSE: 2.213758431322581


<a id="upload"></a>

## 5. Persist the model and custom library

In this section, using `ibm-watsonx-ai` SDK, you will ...
- save the library `linalgnorm-0.1.zip` in WML Repository by creating a package extension resource
- create a Software Specification resource and bind the package resource to it. This Software Specification resource will be used to configure the online deployment runtime environment for a model 
- bind Software Specification resource to the model and save the model to WML Repository

### Create package extension

Define the meta data required to create package extension resource. <br>

The value for `file_path` in `client.package_extensions.LibraryMetaNames.store()` contains the library file name that must be uploaded to the WML.

**Note:** You can also use `requirements.txt` file as package extension input. In such case set the `TYPE` to `requirements_txt` and `file_path` to the `txt` file.
```
client.package_extensions.ConfigurationMetaNames.TYPE = "requirements_txt"
```

In [26]:
meta_prop_pkg_extn = {
    client.package_extensions.ConfigurationMetaNames.NAME: "K_Linag_norm_skl",
    client.package_extensions.ConfigurationMetaNames.DESCRIPTION: "Pkg extension for custom lib",
    client.package_extensions.ConfigurationMetaNames.TYPE: "pip_zip",
}

pkg_extn_details = client.package_extensions.store(
    meta_props=meta_prop_pkg_extn, file_path="linalgnorm-0.1.zip"
)
pkg_extn_id = client.package_extensions.get_id(pkg_extn_details)
pkg_extn_url = client.package_extensions.get_href(pkg_extn_details)

Creating package extensions
SUCCESS


Display the details of the package extension resource that was created in the above cell.

In [27]:
details = client.package_extensions.get_details(pkg_extn_id)

### Create software specification and add custom library

Define the meta data required to create software spec resource and bind the package. This software spec resource will be used to configure the online deployment runtime environment for a model.

In [28]:
client.software_specifications.ConfigurationMetaNames.show()

---------------------------  ----  --------  --------------------------------
META_PROP NAME               TYPE  REQUIRED  SCHEMA
NAME                         str   Y
DESCRIPTION                  str   N
PACKAGE_EXTENSIONS           list  N
SOFTWARE_CONFIGURATION       dict  N         {'platform(required)': 'string'}
BASE_SOFTWARE_SPECIFICATION  dict  Y
---------------------------  ----  --------  --------------------------------


#### List base software specifications

In [29]:
client.software_specifications.list()

,NAME,ID,TYPE,STATE,REPLACEMENT
0,autoai-kb_rt25.1-py3.12,046f5252-0d07-5c23-81ef-18a1a31cdce3,base,supported,
1,watsonx-cfm-caikit-1.0,0cee3c55-472f-57b1-84bd-72f5d066dbe4,base,supported,
2,watsonx-textgen-fm-1.0,129aec82-7e65-5c78-b812-4c0a74b916f5,base,supported,
3,masking-flows-spark,13666829-5570-53a7-927b-52d42a101d93,base,supported,
4,autoai-ts_rt25.1-py3.12,1c0c9e1d-8523-520f-867e-102173f16f5e,base,supported,
5,tensorflow_rt24.1-py3.11,2c33167d-b11c-5490-a305-3e5e95db5c4d,base,supported,
6,pytorch-onnx_rt24.1-py3.11,2da185aa-eac3-59a5-bb8e-0e5b60458a15,base,supported,
7,tensorflow_rt24.1-py3.11-edt,2f53b747-c01a-58a6-a20c-8390ecfd8625,base,supported,
8,kernel-spark3.5-py3.11,30ad99e2-7cfb-511a-9a3e-4cc308071fd3,base,supported,
9,kernel-spark3.5-r4.3,3668b4d5-b8b8-5926-badf-1134137613d4,base,supported,


#### Select base software specification to extend

In [30]:
base_sw_spec_id = client.software_specifications.get_id_by_name("runtime-25.1-py3.12")

#### Define new software specification based on base one and custom library

In [31]:
meta_prop_sw_spec = {
    client.software_specifications.ConfigurationMetaNames.NAME: "linalgnorm-0.1",
    client.software_specifications.ConfigurationMetaNames.DESCRIPTION: "Software specification for linalgnorm-0.1",
    client.software_specifications.ConfigurationMetaNames.BASE_SOFTWARE_SPECIFICATION: {
        "guid": base_sw_spec_id
    },
}

sw_spec_details = client.software_specifications.store(meta_props=meta_prop_sw_spec)
sw_spec_id = client.software_specifications.get_id(sw_spec_details)


client.software_specifications.add_package_extension(sw_spec_id, pkg_extn_id)

SUCCESS


'SUCCESS'

### Save the model

Define the metadata to save the trained model to watsonx.ai Runtime along with the information about the software spec resource required for the model. 

The `client.repository.ModelMetaNames.SOFTWARE_SPEC_ID` metadata property is used to specify the ID of the software spec resource that needs to be associated with the model.

In [32]:
model_props = {
    client.repository.ModelMetaNames.NAME: "Temp prediction model with custom lib",
    client.repository.ModelMetaNames.TYPE: "scikit-learn_1.6",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_ID: sw_spec_id,
}

Save the model to the watsonx.ai Runtime and display its saved metadata. 

In [33]:
published_model = client.repository.store_model(
    model=skl_pipeline, meta_props=model_props
)

In [34]:
published_model_id = client.repository.get_model_id(published_model)
model_details = client.repository.get_details(published_model_id)
print(json.dumps(model_details, indent=2))

{
  "metadata": {
    "name": "Temp prediction model with custom lib",
    "space_id": "8a13841b-df99-4b4d-bf2a-161ad2e33980",
    "resource_key": "5f64edb3-2ea3-444b-a4ca-a40016632676",
    "id": "d0e0164c-0cbc-4288-b3a7-6169a96d3368",
    "created_at": "2025-05-07T10:06:36Z",
    "rov": {
      "member_roles": {
        "1000331001": {
          "user_iam_id": "1000331001",
          "roles": [
            "OWNER"
          ]
        }
      }
    },
    "owner": "1000331001"
  },
  "entity": {
    "software_spec": {
      "id": "3114fcfc-f3f8-4b80-b719-682234b1d232"
    },
    "type": "scikit-learn_1.6"
  }
}


<a id="deploy"></a>

## 6 Deploy and Score

In this section, you will deploy the saved model that uses the custom transformer and perform predictions. You will use WML client to perform these tasks.

### Deploy the model

In [35]:
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Deployment of custom lib model",
    client.deployments.ConfigurationMetaNames.ONLINE: {},
}

created_deployment = client.deployments.create(published_model_id, meta_props=metadata)



######################################################################################

Synchronous deployment creation for id: 'd0e0164c-0cbc-4288-b3a7-6169a96d3368' started

######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
......
ready


-----------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_id='6fba4930-ad32-46bd-842e-da7f96d8a62c'
-----------------------------------------------------------------------------------------------




<a id="score"></a>
### Predict using the deployed model

**Note**: Here we use deployment `id` saved in published_model object. In next section, we show how to retrive deployment url from watsonx.ai instance.

In [36]:
deployment_id = client.deployments.get_id(created_deployment)

Now you can print an online scoring endpoint. 

In [ ]:
scoring_endpoint = client.deployments.get_scoring_href(created_deployment)
print(scoring_endpoint)

Prepare the payload for prediction. The payload contains the input records for which predictions has to be performed.

In [38]:
scoring_payload = {
    "input_data": [{"fields": ["time", "humidity"], "values": [[79863, 47]]}]
}

Execute the method to perform online predictions and display the prediction results

In [39]:
predictions = client.deployments.score(deployment_id, scoring_payload)

In [40]:
print(json.dumps(predictions, indent=2))

{
  "predictions": [
    {
      "fields": [
        "prediction"
      ],
      "values": [
        [
          14.629242312262988
        ]
      ]
    }
  ]
}


<a id="cleanup"></a>
## 7. Clean up 

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watsonx-ai-samples/blob/master/cpd5.2/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>

## 8. Summary

You successfully completed this notebook! 
 
You learned how to use a scikit-learn model with custom transformer in watsonx.ai Runtime service to deploy and score.

Check out our _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

## Author

**Krishnamurthy Arthanarisamy**, is a senior technical lead in IBM watsonx.ai team. Krishna works on developing cloud services that caters to different stages of machine learning and deep learning modeling life cycle.

**Lukasz Cmielowski**, PhD, is a Software Architect and Data Scientist at IBM.

Copyright © 2020-2025 IBM. This notebook and its source code are released under the terms of the MIT License.